In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/gregory_maruss/VESUVIUS_Challenge/jupyter notebooks
Current path:/home/gregory_maruss/VESUVIUS_Challenge


In [2]:
import torch
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Tile_Dataset import Vesuvius_Tile_Datamodule
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
import einops
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

2023-05-04 13:37:16,460 - Created a temporary directory at /tmp/tmp0fcm279x
2023-05-04 13:37:16,461 - Writing /tmp/tmp0fcm279x/_remote_module_non_scriptable.py


In [3]:
class CFG:
    
    train_fragment_id=[2,3]
    val_fragment_id=[1]
    batch_size = 24
    patch_size = 224
    z_dim = 32
    stride = patch_size // 2
    #comp_dataset_path = COMPETITION_DATA_DIR
    num_workers = 8
    on_gpu = True

In [4]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [5]:
lit_model = UNET_TILE_lit(
        #wandb = True,
        z_dim = 32,
        patch_size = (224,224),
        sw_batch_size=8 ,
        eta_min = 1e-7,
        t_max = 100,
        max_epochs = 1000,
        weight_decay =  0.001,
        learning_rate = 0.00007,
        gamma = 0.85,)

FROM_CHECKPOINT = False
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/Full_16_768to1024_batch/FocalDice_768epoch=168FBETA=0.54recall=0.45precision=0.57.ckpt', 
                                             #patch_size = (512,512),
                                             learning_rate = 0.0001,
                                              weight_decay =  .001,
                                              eta_min = 1e-7,
                                             sw_batch_size = 8)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [ ]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=7,
    monitor="fbeta_4",
    mode="max",
    dirpath="logs/SMPEffB4_Tile_32_224patch/",
    filename="SMPEffB4_Tile_32_224patch{epoch:02d}{val_loss:.2f}{fbeta_4:.2f}{fbeta_6:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=100,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/SMPEffB4_Tile_32_224patch/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/SMPEffB4_Tile_32_224patch/',
        #overfit_batches=1,
        precision=16,
        accumulate_grad_batches=1, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Full_16_768to1024_batch/FocalDice_768epoch=168FBETA=0.54recall=0.45precision=0.57.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-04 13:39:28,191 - Using 16bit None Automatic Mixed Precision (AMP)
2023-05-04 13:39:28,244 - GPU available: True (cuda), used: True
2023-05-04 13:39:28,245 - TPU available: False, using: 0 TPU cores
2023-05-04 13:39:28,245 - IPU available: False, using: 0 IPUs
2023-05-04 13:39:28,246 - HPU available: False, using: 0 HPUs
2023-05-04 13:39:30,854 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Adjusting learning rate of group 0 to 1.0000e-04.
2023-05-04 13:39:30,871 - 
  | Name         | Type                  | Params
-------------------------------------------------------
0 | metrics      | ModuleDict            | 0     
1 | model        | Unet                  | 20.2 M
2 | loss_dice    | DiceLoss              | 0     
3 | loss_tversky | TverskyLoss           | 0     
4 | loss_bce     | SoftBCEWithLogitsLoss | 0     
-------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
40.476    Total estimated model

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.9975e-05.


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.2224e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.1363e-05.


Validation: 0it [00:00, ?it/s]